In [1]:
# Import of the necessary libraries
import tweepy
import json
import pandas as pd
import datetime

# http://docs.tweepy.org/en/latest/api.html

We read the api keys from our config json file 

This is done so we do not expose our keys in github

In [6]:
# Values assigning for the authentication methods     
consumer_key = "oeRyEcKV5Ebfu7vPZVvtTs45H"
consumer_secret = "LNtBLVDspKep0WJ6XI7fceoT81KpSr3u51BefUUu44tOZ6rawT"
access_token = "1299838566827274242-vPAr4kIzeFfKXtnPc8TjpbFWAi8TVS"
access_token_secret = "NH4DvtNlS41YCaq0gsOdSOmqw6PWrvMv76oizYWjuxHCj"

Run the authentication methods

In [43]:
def get_api_connection(consumer_key, consumer_secret, access_token, access_token_secret):
  # Setting access tokens
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)

  # Create the API object from tweepy
  api = tweepy.API(auth, wait_on_rate_limit=True)
  return api




Define functions to handle tweepy objects easily

In [44]:
# Given a query, it returns a number of tweet objects
# We give a default of 100 but any number could be used
def get_tweets_from_query(api, query, number_of_tweets_to_get = 100):
    return tweepy.Cursor(api.search, q=query).items(number_of_tweets_to_get)  

# Given a tweet obj we return its number of retweets
def get_rts_from_tweet(api, tweet):
    return api.get_status(tweet.id).retweet_count

# Given a tweet obj we return its username
def get_user_from_tweet(api, tweet):
    return api.get_status(tweet.id).user.screen_name

# Given a tweet obj we return its likes/favs number
def get_likes_from_tweet(api, tweet):
    return api.get_status(tweet.id).favorite_count

In [45]:
# Class for better analysis with only the needed parameters
class Tweet_for_analysis:
    text: ""
    rts: 0
    id: 0
    user: ""
    likes: 0
    time: datetime.datetime.now

    # Constructor
    def __init__(self, api, tweetObj):
        self.text = tweetObj.text
        self.rts =  get_rts_from_tweet(api, tweetObj)
        self.id = tweetObj.id
        self.user = get_user_from_tweet(api, tweetObj)
        self.likes = get_likes_from_tweet(api, tweetObj)
        self.time = tweetObj.created_at



In [46]:
# Given the twitter keys, a query and the number of tweets to return
# returns a dataset
def get_tweet_df_from_query(twitter_consumer_key, 
                            twitter_consumer_secret, 
                            twitter_access_token, 
                            twitter_access_token_secret, 
                            query, 
                            n_tweets = 50
                            ):
  api = get_api_connection(consumer_key,consumer_secret,access_token,access_token_secret)
  new_tweet_list = []
  tweets_received = get_tweets_from_query(api, query, n_tweets)
  for tweet in tweets_received:
    new_tweet_to_add = Tweet_for_analysis(api, tweet)
    new_tweet_list.append(new_tweet_to_add)
  
  df = pd.DataFrame(
    ([query, tweet.id, tweet.text, tweet.rts, tweet.user, tweet.likes, tweet.time] for tweet in new_tweet_list), 
    columns = ["query", "tweet_id","text","rt_count", "username", "likes_count", "created_at"]
  )

  return df

In [50]:
df_pd = get_tweet_df_from_query(consumer_key,consumer_secret,access_token,access_token_secret,"policías negacionistas",5000)

In [51]:
df_pd.tail()

,query,tweet_id,text,rt_count,username,likes_count,created_at
2261,policías negacionistas,1321577469326041088,"Policías locales amonestando a ciudadanos ""por...",6,Hipogresia155,10,2020-10-28 22:19:57
2262,policías negacionistas,1321154506994950145,RT @gonzo_blogger: ÚLTIMA HORA URGENTE: Los ca...,131,Negacionistas,0,2020-10-27 18:19:14
2263,policías negacionistas,1320817403182391297,@JuanCLongas El momento en que se toman las me...,0,Naburro4,2,2020-10-26 19:59:43
2264,policías negacionistas,1320710074541740032,RT @_fransua: varias comunidades autonomas com...,1,platini511,0,2020-10-26 12:53:14
2265,policías negacionistas,1320708713662062594,varias comunidades autonomas como Catalunya; A...,1,_fransua,3,2020-10-26 12:47:49


Save data

In [49]:
#Create an appropiate folder

hora_madrid_delta = 2

anadir_hora = datetime.timedelta(hours = hora_madrid_delta)

now = datetime.datetime.now()
now = now+anadir_hora#local time
now= now.strftime("%d_%m_%Y__%Hh_%Mm_%Ss")
now_hour=now[:15]


import os

path = "/content/drive/My Drive/Colab Notebooks/UOC_B/twitter_tweets_policias_negacionistas_"+str(now_hour)

if not os.path.exists(path):
    os.makedirs(path)

#Write dataframe to csv
df_pd.to_csv(path+"/twitter_tweets_"+str(now)+".csv", index = False)